In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from DifferentialEvolution import DifferentialEvolution
import plotly.graph_objs as go
from plotly.subplots import make_subplots

%load_ext autoreload
%autoreload 2

In [2]:
LIMITS = [(-10,10)]*2

PARAMS = {
    "popsize":30,
    "iterations":100,
    "mut":0.5,
    "crossp":0.7,
    "generation_history":True,
    "track_best":True,
}

def squares(x):
    return sum(i*i for i in x)

currentToBest = DifferentialEvolution(squares, LIMITS)
randMutation = DifferentialEvolution(squares, LIMITS)

In [3]:
INITIAL_POPULATION = 1/10 * np.mgrid[1:10,1:10]
INITIAL_POPULATION = np.rec.fromarrays(INITIAL_POPULATION).flatten()
INITIAL_POPULATION = np.array([np.array(list(pt)) for pt in INITIAL_POPULATION])

In [24]:
currentToBest.optimize(**PARAMS, initial_pop=INITIAL_POPULATION.copy())
randMutation.optimize(**PARAMS, initial_pop=INITIAL_POPULATION.copy())

(array([0., 0.]), 0.0)

In [4]:
def make_mesh(limits):
    x0, x1 = limits[0]
    y0, y1 = limits[1]
    
    x = np.arange(x0, x1 + 0.1, 0.5)
    y = np.arange(y0, y1 + 0.1, 0.5)
    
    xs, ys = np.meshgrid(x,y)
    return xs, ys, x, y

In [59]:
xs, ys, x, y = make_mesh(LIMITS)

z = xs**2 + ys**2

fig = make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': False}]],)
fig.add_trace(
    go.Surface(
        x=x,y=y,z=z, opacity=0.3,showscale=False
        ),
    1,1
)
fig.add_trace(
    go.Contour(
        z=z, opacity=0.3, showscale=False, x0=-10, dx=0.5, y0=-10, dy=0.5
        ),
    1,2
)

for i, gen in enumerate(currentToBest.generation_history):
    if i < 20 and (i < 2 or i % 5 == 0):
        ptsx = np.array([x for x, _ in gen])
        ptsy = np.array([y for _, y in gen])
        ptsz = ptsx**2 + ptsy**2
        fig.add_trace(
            go.Scatter3d(
                x=ptsx,
                y=ptsy, 
                z=ptsz, 
                mode='markers',
                marker=dict(size=4),
                name=f'Generation {i}'
            ), 1, 1
        )
        fig.add_trace(
            go.Scatter(
                x=ptsx,
                y=ptsy, 
                #z=ptsz, 
                mode='markers',
                marker=dict(size=6),
                name=f'Generation {i}'
            ), 1, 2
        )

        

fig.update_layout(width=1400, height=700, showlegend=True)
fig.show()

In [11]:
import math
def ackley_function(x1, x2):
    x1, x2
    part_1 = -0.2*math.sqrt(0.5*(x1*x1 + x2*x2))
    part_2 = 0.5*(math.cos(math.pi*x1) + math.cos(math.pi*x2))
    value = math.exp(1) + 10 -10*math.exp(part_1) - math.exp(part_2)
    return value


ackleyOptimization = DifferentialEvolution(lambda x: ackley_function(*x), LIMITS)
ackleyOptimization.optimize(**PARAMS, initial_pop=INITIAL_POPULATION.copy())

(array([0., 0.]), -4.440892098500626e-16)

In [13]:
xs, ys, x, y = make_mesh(LIMITS)

vect_ackley = np.vectorize(ackley_function)

z = vect_ackley(xs, ys)

fig = make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': False}]],)
fig.add_trace(
    go.Surface(
        x=x,y=y,z=z, opacity=0.5,showscale=False
        ),
    1,1
)
fig.add_trace(
    go.Contour(
        z=z, opacity=0.3, showscale=False, x0=-10, dx=0.5, y0=-10, dy=0.5
        ),
    1,2
)

for i, gen in enumerate(ackleyOptimization.generation_history):
    if i < 20 and (i < 2 or i % 5 == 0):
        ptsx = np.array([x for x, _ in gen])
        ptsy = np.array([y for _, y in gen])
        ptsz = vect_ackley(ptsx, ptsy)
        fig.add_trace(
            go.Scatter3d(
                x=ptsx,
                y=ptsy, 
                z=ptsz, 
                mode='markers',
                marker=dict(size=8),
                name=f'Generation {i}'
            ), 1, 1
        )
        fig.add_trace(
            go.Scatter(
                x=ptsx,
                y=ptsy, 
                #z=ptsz, 
                mode='markers',
                marker=dict(size=6),
                name=f'Generation {i}'
            ), 1, 2
        )

        

fig.update_layout(width=1400, height=700, showlegend=True)
fig.show()